In [ ]:
# Review Exercises
import numpy as np

# • Define Arrow-Debreu securities and their payoff vectors in R^2
# Arrow-Debreu securities pay 1 in one state and 0 in the other (state-contingent claims)
# For two states (up and down):
AD_up = np.array([1, 0]) # Pays 1 in up state, 0 in down
AD_down = np.array([0, 1]) # Pays 1 in down state, 0 in up
print(f"Arrow-Debreu up: {AD_up}")
print(f"Arrow-Debreu down: {AD_down}")

# • Given bond and stock, compute payoff matrix M and check invertibility
# Bond B1 pays 1 in both states (risk-free)
B1 = np.array([1, 1])
# Stock S1 payoffs in up and down states
S1 = np.array([120, 80])
# Payoff matrix M with columns as asset payoffs
M = np.column_stack((B1, S1))
print(f"Payoff matrix M:\n{M}")

# Check if invertible (determinant != 0)
det_M = np.linalg.det(M)
is_invertible = det_M != 0
print(f"Determinant: {det_M}, Invertible: {is_invertible}")

# • Given claim payoff C1, compute replication portfolio phi = M^{-1} C1 and arbitrage price C0 = phi · M0 (dot product with t=0 prices)
C1 = np.array([100, 60]) # Example claim payoff in up and down
if is_invertible:
    phi = np.linalg.inv(M) @ C1 # Or np.linalg.solve(M, C1)
    print(f"Replication portfolio phi: {phi}")
else:
    print("M not invertible, cannot replicate exactly.")

# Assume t=0 prices M0 = [B0, S0]
B0 = 0.95 # Bond price
S0 = 100 # Stock price
M0 = np.array([B0, S0])
C0 = np.dot(phi, M0)
print(f"Arbitrage-free price C0: {C0}")


In [ ]:
# Exercises

# • Compute state-price vector psi from complete market and verify M0 = M^T psi
# Using previous M (payoff matrix), M0 (t=0 prices)
B1 = np.array([1, 1])
S1 = np.array([120, 80])
M = np.column_stack((B1, S1))
B0 = 0.95
S0 = 100
M0 = np.array([B0, S0])

# State prices psi solve M^T psi = M0 (since prices are discounted expected payoffs under psi)
# psi = (M^T)^{-1} M0, but since M is square and invertible
psi = np.linalg.solve(M.T, M0)
print(f"State-price vector psi: {psi}")

# Verify M^T psi == M0
verified = np.allclose(M.T @ psi, M0)
print(f"Verification M^T psi = M0: {verified}")

# • Derive risk-neutral q from psi and r, verify C0 = 1/(1+r) E_Q[C1]
# Short rate r from bond: B0 = 1 / (1 + r) => r = 1/B0 - 1
r = 1 / B0 - 1
print(f"Short rate r: {r}")

# Risk-neutral q = (1 + r) * psi_up (for up state), and 1 - q for down
q = (1 + r) * psi[0] # Since psi = q/(1+r) for up, (1-q)/(1+r) for down
print(f"Risk-neutral probability q: {q}")
# Check normalization: sum(psi) * (1 + r) should be 1 (martingale measure)
print(f"Check q + (1-q) = 1: {q + (1 + r) * psi[1]}")  # Should be ~1

# Verify for a claim C1
C1 = np.array([100, 60])
E_Q_C1 = q * C1[0] + (1 - q) * C1[1]
C0_rn = E_Q_C1 / (1 + r)
# Compare to previous C0 from replication
phi = np.linalg.solve(M, C1)
C0_rep = np.dot(phi, M0)
print(f"Risk-neutral C0: {C0_rn}, Matches replication: {np.allclose(C0_rn, C0_rep)}")

# • Construct “too cheap” claim price and exhibit arbitrage strategy
# Assume fair C0 from above, but observed C0_obs < C0 (too cheap)
C0_fair = C0_rn
C0_obs = C0_fair - 0.5  # Example: 0.5 too cheap

# Arbitrage: Buy the cheap claim, sell the replicating portfolio
# Strategy: Buy 1 claim at C0_obs, short phi units of assets
arb_initial = C0_fair - C0_obs
print(f"Arbitrage initial inflow: {arb_initial}")

# At t=1, claim pays C1, replicating portfolio pays M phi = C1. Net payoff: C1 - C1 = 0 in all states
# Risk-free profit = arb_initial
print("Arbitrage strategy: Buy 1 claim, short the replication portfolio phi.")
print(f"Net t=0 cash: +{arb_initial}, Net t=1 payoff: 0 in all states.")

In [4]:
# Capstone
# • Reusable pricing function
def two_state_pricer(M, M0, C1, tol=1e-8):
    """
    Price a claim C1 in two-state economy.
    - M: 2x2 payoff matrix (columns: asset payoffs)
    - M0: array of t=0 asset prices
    - C1: claim payoff vector (2,)
    Returns: phi (replication portfolio), C0 (price), or raises error if not replicable.
    """
    # • Sanity check: Detect singular M
    det_M = np.linalg.det(M)
    if abs(det_M) < tol:
        # Near-singular: Use least-squares instead of solve
        print("Warning: M near-singular, using least-squares approximation.")
        phi, residuals, _, _ = np.linalg.lstsq(M, C1, rcond=None)
        replicated = M @ phi
        residual_norm = np.linalg.norm(residuals)
        if residual_norm > tol:
            raise ValueError(f"Cannot replicate: residual norm {residual_norm} > tol.")
    else:
        # Invertible: Use solve
        phi = np.linalg.solve(M, C1)
        replicated = M @ phi
        residual_norm = np.linalg.norm(replicated - C1)
        if residual_norm > tol:
            raise ValueError(f"Replication failed: residual norm {residual_norm} > tol.")

    # Price C0 = phi · M0
    C0 = np.dot(phi, M0)
    
    # Verify replication
    print(f"Replication residual norm: {residual_norm}")
    return phi, C0

# Example usage
B1 = np.array([1, 1])
S1 = np.array([120, 80])
M = np.column_stack((B1, S1))
M0 = np.array([0.95, 100])
C1 = np.array([100, 60])

phi, C0 = two_state_pricer(M, M0, C1)
print(f"Phi: {phi}, C0: {C0}")

# • Stress test: Generate 100 random claims, check linearity p(C1 + C1') = p(C1) + p(C1')
np.random.seed(42)
for _ in range(100):
    C1_a = np.random.uniform(50, 150, 2)
    C1_b = np.random.uniform(50, 150, 2)
    C1_sum = C1_a + C1_b
    
    _, C0_a = two_state_pricer(M, M0, C1_a)
    _, C0_b = two_state_pricer(M, M0, C1_b)
    _, C0_sum = two_state_pricer(M, M0, C1_sum)
    
    assert np.allclose(C0_a + C0_b, C0_sum, atol=1e-6), "Linearity failed!"
print("Stress test passed: Linearity confirmed for 100 random claims.")

# • Test near-singular case (for hint)
M_singular = np.column_stack((B1, np.array([100, 100])))
try:
    two_state_pricer(M_singular, M0, C1)
except ValueError as e:
    print(f"Near-singular test: {e}")

Replication residual norm: 0.0
Phi: [-20.   1.], C0: 81.0
Replication residual norm: 4.0194366942304644e-14
Replication residual norm: 0.0
Replication residual norm: 0.0
Replication residual norm: 0.0
Replication residual norm: 6.355287432313019e-14
Replication residual norm: 0.0
Replication residual norm: 0.0
Replication residual norm: 2.1316282072803006e-14
Replication residual norm: 6.355287432313019e-14
Replication residual norm: 0.0
Replication residual norm: 0.0
Replication residual norm: 0.0
Replication residual norm: 0.0
Replication residual norm: 0.0
Replication residual norm: 0.0
Replication residual norm: 0.0
Replication residual norm: 0.0
Replication residual norm: 0.0
Replication residual norm: 0.0
Replication residual norm: 2.0097183471152322e-14
Replication residual norm: 0.0
Replication residual norm: 1.4210854715202004e-14
Replication residual norm: 1.4210854715202004e-14
Replication residual norm: 0.0
Replication residual norm: 2.1316282072803006e-14
Replication resid